In [1]:
#將句子做分類，查看他屬於哪一個主題
from gensim.models import Word2Vec
from gensim import models

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
model = models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
from sklearn.neural_network import MLPClassifier

In [5]:
df = pd.read_csv('250google_vector.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,class,word
0,-0.181641,0.165039,-0.166016,0.357422,-0.092285,0.201172,-0.054688,-0.261719,-0.172852,-0.080566,...,-0.149414,0.398438,0.092773,0.033936,-0.061523,0.159180,0.127930,0.002228,0,food
1,0.067871,-0.041260,0.100098,0.058838,0.054443,0.017578,0.072266,-0.091309,0.072266,0.122559,...,-0.150391,0.175781,0.106445,0.157227,-0.113281,0.104492,-0.085449,-0.154297,1,service
2,0.121094,0.281250,0.106445,0.025635,-0.112305,0.040527,-0.006622,-0.172852,-0.363281,0.142578,...,-0.064941,0.074707,0.118164,0.185547,0.248047,-0.273438,0.072754,0.013489,2,ambience
3,0.079590,0.018799,0.109375,0.018677,-0.114258,0.120117,0.172852,-0.020630,0.306641,0.267578,...,-0.166016,-0.080078,0.144531,0.398438,0.064453,-0.066406,0.073242,-0.188477,3,value
4,-0.026855,0.081055,-0.171875,0.227539,-0.141602,0.040527,0.229492,-0.227539,-0.332031,0.131836,...,-0.174805,-0.055176,0.197266,0.199219,-0.361328,0.101074,0.142578,0.281250,0,foodstuff


In [6]:
X_train = df.iloc[:,0:300].values.tolist()
y_train = df.iloc[:,300].values.tolist()

In [7]:
from collections import Counter

def most_frequent(List):
    b = Counter(List)
    z = max(b.values())
    return [f for f in b.keys() if b[f] == z]

In [8]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(42), random_state=1)
clf.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=42, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [9]:
df = pd.read_csv('names.csv', header=None)
# col_names = pd.read_csv('names.csv', header=None).values.tolist()
tuples = [tuple(x) for x in df.values]
# tuples
words = list()
for t in tuples:
    for x in t:
#         print(x)
        words.append(x)   

In [10]:
i=0
food_list=list()
service_list=list()
ambience_list=list()
value_list=list()

for x in words:
    if(i%4==0):
        food_list.append(x)
    elif(i%4==1):
        service_list.append(x)
    elif(i%4==2):
        ambience_list.append(x)
    elif(i%4==3):
        value_list.append(x)

    i+=1
food_mean = np.mean(model[food_list], axis=0)
service_mean = np.mean(model[service_list], axis=0)
ambience_mean = np.mean(model[ambience_list], axis=0)
value_mean = np.mean(model[value_list], axis=0)
#cosine_similarity = np.dot(x, y)/(np.linalg.norm(x)* np.linalg.norm(y))

In [45]:
topic = list()

threshold = 0.25 #經過測試使用0.25
reviews = 1324 #要隨著reviews筆數更改
for x in range(1,reviews+1): #總共1~reviews+1 
    df = pd.read_csv('review_sets/review'+str(x)+'.txt', sep=" ", header=None)
    #df = pd.read_csv('datasets/dataset'+str(x)+'.txt', sep=" ", header=None)
    df = df.dropna(axis=1)
    words = df.values.tolist()[0]
    words_vec = list()
    count = 0
    #y = np.array([])
    y = list()
    for z in words:#可能會有不再googlenews的字 要除外
        if(z=='ExperienceLess'):
            z='Experience'
        if(z=='dineVery'):
            z='dine'
        if(z=='grunerveltliner'):
            z='wine'    
        if(z=='mignardises'):
            z='snack'
        if(z=='friendlyBooked'):
            z='friendly'
        if(z=='Nickquae'):
            z='Nick'
        if(z=='interchangeableOverall'):
            z='interchangeable'
        if(z=='cakestand'):
            z='cake'
        if(z=='againLinguini'):
            z='again'
        if(z=='tripadvisor'):
            z='website'
        if(z=='PICCOLA'):
            z='small'
        a=model[z]
        #cosine_similarity = np.dot(a, b)/(np.linalg.norm(a)* np.linalg.norm(b))
        c_s_f = np.dot(a, food_mean)/(np.linalg.norm(a)* np.linalg.norm(food_mean))
        c_s_s = np.dot(a, service_mean)/(np.linalg.norm(a)* np.linalg.norm(service_mean))
        c_s_a = np.dot(a, ambience_mean)/(np.linalg.norm(a)* np.linalg.norm(ambience_mean))
        c_s_v = np.dot(a, value_mean)/(np.linalg.norm(a)* np.linalg.norm(value_mean))
        #檢查是否小於門檻值
        if(c_s_f < threshold and c_s_s < threshold and c_s_a < threshold and c_s_v < threshold):
            count+=1
        else:
            words_vec.append(model[z])
    if(len(words_vec)!=0):#防止words_vec是空的
        X = words_vec
        y = clf.predict(X)
    for i in range(count):#在列表中依照count次數加入4
        y=np.append(y,4)
    #topic.append(most_frequent(list(y)))
    topic.append(most_frequent(y))
    #topic.append(y)

In [46]:
print(topic)

[[4.0], [2, 3], [1], [4], [2], [4], [3], [4], [4], [3], [4], [4], [1], [1], [4], [4], [2, 4], [4], [4], [0, 4], [3], [0], [0], [1, 0, 4], [2], [4], [0, 2, 4], [4], [4], [1], [4], [4], [3], [0], [3], [0], [2, 0, 3], [4], [2], [2, 4], [4.0], [0], [4.0], [4.0], [4.0], [4.0], [4], [3, 4], [0], [4], [3, 2, 4], [0, 3], [4.0], [4.0], [1], [4.0], [1, 3, 4], [4], [4], [1, 3, 4], [1], [4.0], [4.0], [0], [1], [4], [4], [3], [4], [4], [0, 4], [3, 1, 4], [0], [4], [4], [4.0], [0, 4], [1, 4], [4], [2, 4], [0, 3], [1, 4], [4.0], [1, 2, 4], [3, 1], [3, 4], [4.0], [3], [1], [4], [4], [2], [4], [4], [1, 2, 4], [4], [4], [1], [4.0], [4], [3], [1], [4], [4.0], [4.0], [4], [2], [4.0], [1], [4.0], [4.0], [4], [0], [0], [1, 2, 4], [3], [3], [0], [4], [4.0], [3], [4.0], [1, 3, 4], [4], [3, 4], [2], [4], [0], [2, 1], [4.0], [3, 0], [4], [0], [1], [2], [4], [0, 4], [4.0], [4], [2, 4], [0], [2, 4], [4], [1], [2], [1, 4], [4], [0, 2], [0, 4], [3, 1, 0, 4], [4], [4], [2], [1], [3], [3], [3, 4], [0, 1, 4], [0, 3, 4

In [47]:
rlist=list()
for i in range(1,reviews+1):
    rlist.append("review"+str(i))
df3 = pd.DataFrame(rlist,columns=["reviews"])
df3.insert(1,'topic',topic)

In [48]:
food=list()
service=list()
ambience=list()
value=list()
others=list()
length = np.shape(df3)[0]
for x in range(length):
    for y in range (len(df3.values[x][1])):
        if(df3.values[x][1][y]==0): #包含food主題的
            food.append(df3.values[x][0])
        elif(df3.values[x][1][y]==1): #包含service主題的
            service.append(df3.values[x][0])
        elif(df3.values[x][1][y]==2): #包含ambience主題的
            ambience.append(df3.values[x][0])
        elif(df3.values[x][1][y]==3): #包含value主題的
            value.append(df3.values[x][0])
        elif(df3.values[x][1][y]==4): #包含others主題的
            others.append(df3.values[x][0])

In [49]:
df = pd.DataFrame(data={0: food})
df.to_csv("./classify_reviews/food_topic.csv",index=False)

df = pd.DataFrame(data={0: service})
df.to_csv("./classify_reviews/service_topic.csv",index=False)

df = pd.DataFrame(data={0: ambience})
df.to_csv("./classify_reviews/ambience_topic.csv",index=False)

df = pd.DataFrame(data={0: value})
df.to_csv("./classify_reviews/value_topic.csv",index=False)

df = pd.DataFrame(data={0: others})
df.to_csv("./classify_reviews/others_topic.csv",index=False)